In [1]:
# !pip install langchain

In [2]:
import os
import openai
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv, find_dotenv
import warnings
warnings.filterwarnings("ignore")
from secret_key import openapi_key

os.environ['OPENAI_API_KEY'] = openapi_key
# load_dotenv(find_dotenv())
# openai.api_key = os.environ["OPENAI_API_KEY"]
# os.environ["OPENAI_API_KEY"]

In [3]:
llm = OpenAI(model_name="gpt-3.5-turbo")
llm("Tell me a joke")

'Why was the math book sad?\n\nBecause it had too many problems.'

## PromptTemplates 
Prompt Templates makes creating a prompt for different usecases easier than using f-strings and interpolate the videos

In [4]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

In [5]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
print(chain.predict(input="I ordered Pizza Salami and it was awesome!"))

{
  "sentiment": "positive",
  "subject": "food"
}


## Real World example with ResponseSchema, Templates, Chains and OutputParsers
There were two issues with the output: The output also contains text and the output is just a string which can not just the converted to a dictionary. Lets make it better with a more complex example

In [6]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

sentiment_schema = ResponseSchema(
    name="sentiment",
    description="Is the text positive, neutral or negative? Only provide these words",
)
subject_schema = ResponseSchema(
    name="subject", description="What subject is the text about? Use exactly one word."
)
price_schema = ResponseSchema(
    name="price",
    description="How expensive was the product? Use None if no price was provided in the text",
)

response_schemas = [sentiment_schema, subject_schema, price_schema]

In [7]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sentiment": string  // Is the text positive, neutral or negative? Only provide these words
	"subject": string  // What subject is the text about? Use exactly one word.
	"price": string  // How expensive was the product? Use None if no price was provided in the text
}
```


In [9]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Just return the JSON, do not add ANYTHING, NO INTERPRETATION!

text: {input}

{format_instructions}

"""

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template(template=template)

format_instructions = parser.get_format_instructions()

messages = prompt.format_messages(
    input="I ordered Pizza Salami for 9.99$ and it was awesome!",
    format_instructions=format_instructions,
)

chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
response

AIMessage(content='```json\n{\n\t"sentiment": "positive",\n\t"subject": "food",\n\t"price": "9.99$"\n}\n```')

In [10]:
output_dict = parser.parse(response.content)
print(output_dict)

{'sentiment': 'positive', 'subject': 'food', 'price': '9.99$'}
